In [1]:
import xml.etree.ElementTree as ET
import os
import sys
sys.path.append("conceptual_spaces")
import cs.cs as space
from cs.weights import Weights
from cs.cuboid import Cuboid
from cs.core import Core
from cs.core import from_cuboids
from cs.concept import Concept
import matplotlib.pyplot as plt
import visualization.concept_inspector as ci



path = "Dataset\\prototypes.xml"
path2 = "Dataset\\exemplars.xml"


def print_database(node,indent=''):
    print(indent,node.tag,end=' ')
    try:
        print(node.attrib['name'],end=' ')
    except KeyError:
        pass
    if len(node):
        print()
        for subelem in node:
            print_database(subelem,indent+"  ")
    else:
        print(node.text)
    
def get_domains(xml_elem):
    domains = xml_elem.find('genericPhysicalDescription')
    if(not(domains)):
        return
    domain_dict={}
    try:
        supercategory = xml_elem.find('family').text
        if supercategory in ['feline','rodent','primate','cetacean']:
            supercategory='mammal'
        elif supercategory in ['crustacean']:
            supercategory = 'arthropod'
        elif supercategory in ['fruit', 'transport', 'furniture', 'book', 'building', 'musical_instrument', 'present', 'architectural_element']:
            supercategory = 'object'
        domain_dict['supercategory']=supercategory
    except AttributeError:
        domain_dict['supercategory']="object"
        
    for domain in domains:
        numerical_domain={}
        for subelem in domain:
            try:
                numerical_domain[subelem.tag]=float(subelem.text)
            except ValueError:
                pass
        if numerical_domain:
            domain_name=domain.tag
            
            if not domain_name=='size':
                pass
            elif(domain_name=='hasPart' or domain_name=='partOf'):
                pass
                '''domain_name = domain.get('name')
                domain_dict['n_'+domain_name]={'number_'+domain_name:value for key,value in zip(numerical_domain.keys(),numerical_domain.values())}'''
            else:
                domain_dict[domain_name]={key:value for key,value in zip(numerical_domain.keys(),numerical_domain.values())}
    return domain_dict

def xml_to_dict(path, exemplars = {}):
    tree = ET.parse(os.path.normpath(path))
    root = tree.getroot()
    for exemplar in root:
        #print(animal.attrib['name'])
        new_concept = get_domains(exemplar)
        if(new_concept):
            exemplars[exemplar.attrib['name']] = new_concept
    return exemplars

def is_consistent(concept):
    for domain in concept:
        if not domain=='supercategory':
            if not(len(concept[domain])==len(space._domains[domain])):
                return False
    return True
                
                
def concepts_into_space(concepts, domains={}, except_for=None, colors=['b','g','r','c','m','brown','rosybrown','darkslategrey','pink','grey']):
    if except_for:
        for concept in list(concepts.keys()):
            if concepts[concept]['supercategory']==except_for:
                del(concepts[concept])
                
    ordered_dict={}
    for example in concepts:
        if concepts[example]['supercategory'] in ordered_dict:
            ordered_dict[concepts[example]['supercategory']].append(example)
        else:
            ordered_dict[concepts[example]['supercategory']]=[example]
    '''for key in ordered_dict:
        print(key)'''
    #ordered_dict={'mammal':ordered_dict['mammal']}
    #print(ordered_dict)
    
    colors=iter(colors)
    concept_colors={}
    if(domains=={}):
        for concept in concepts.values():
            domains.update(concept)
        del(domains['supercategory'])
    #domains = mapping: domain->dimension indices
    domain_mapping = {}
    i=0
    for key in domains:
        domain_mapping[key]=list(range(i,i+len(domains[key])))
        i+=len(domains[key])

    dimension_names = []
    for domain in domains.values():
        dimension_names += [dim for dim in domain.keys()]
    space.init(len(dimension_names),domain_mapping,dimension_names)
    
    for category in ordered_dict:
        print('putting',category,'into space')
        cuboids=[]
        domains = {}
        for example in ordered_dict[category]:
            if not is_consistent(concepts[example]):
                print(example,' is inconsistent')
            else:
                subdomains={domain:space._domains[domain] for domain in concepts[example] if not domain == 'supercategory'}
                point=[concepts[example].get(domain,{str(key):float("inf") for key in range(len(space._domains[domain]))}).values() for domain in space._domains]
                point=[p for dom in point for p in dom]
                #print(example)
                #print('point: ',point)
                cuboids.append(Cuboid([dim if not dim==float('inf') else -dim for dim in point], point, subdomains))
                domains.update(subdomains)
                #print(cuboids[-1]._p_max)
        #for cuboid in cuboids:
            #print('cuboid:',cuboid)
        core=from_cuboids(cuboids,domains)
        weights=Weights(space._def_dom_weights,space._def_dim_weights)
        concept=Concept(core, 1.0, 0.5, weights)
        space.add_concept(category,concept,next(colors))
        
    animal_concept=[]
    for concept in space._concepts.values():
        animal_concept+=concept._core._cuboids
    #print('animal cubs:\n',animal_concept)
    core=from_cuboids(animal_concept,space._domains)
    weights=Weights(space._def_dom_weights,space._def_dim_weights)
    concept=Concept(core, 1.0, 0.5, weights)
    space.add_concept('animal',concept,next(colors))
        
        
"""    for example in concepts:
        if concepts[example]['supercategory'] in ['mammal','reptile','bird','amphibian','insect']:
            consistent = True
            domains = {}
            for key in concepts[example]:
                if(not(key=='supercategory')):
                    domains[key]=space._domains[key]

            '''dimension_names = []

            for concept_domain in concepts[example].values():
                dimension_names += [dim for dim in domain.keys()]'''

            dimension_values=[]
            for domain in space._domains.keys():
                dimension_values.append([list(concepts[example].get(domain,{str(key):float("-inf") for key in range(len(space._domains[domain]))}).values())
                                     ,list(concepts[example].get(domain,{str(key):float("inf") for key in range(len(space._domains[domain]))}).values())])
                if(not(len(dimension_values[-1][0])==len(space._domains[domain]))):
                    #print(domain+' in '+example+' is inconsistent')
                    consistent=False
                    break
                if(not(consistent)):
                    print('bah')


            if(consistent):
                p_min=[value for domain in dimension_values for value in domain[0]]
                p_max=[value for domain in dimension_values for value in domain[1]]

                #try:
                c_example = Cuboid(p_min, p_max, domains)
                s_example = Core([c_example], domains)
                w_example = Weights(space._def_dom_weights,space._def_dim_weights)
                concept = Concept(s_example, 1.0, 0.5, w_example)

                supercategory = concepts[example]['supercategory']
                if(supercategory in space._concepts):
                    space.add_concept(supercategory, space._concepts[supercategory].union_with(concept), concept_colors[supercategory])
                else:
                    concept_colors.update({supercategory:next(colors)})
                    space.add_concept(supercategory, concept, concept_colors[supercategory])"""
                #print('added ', example)
                #print(concepts[example]['supercategory'])
        #except Exception:
        #    print(example,'is inconsistent')


def one_shot(point, supercategory='none'):
    #todo find lowest concept
    '''print(point)'''
    for concept in space._concepts:
        '''print(concept)
        print('membership:')
        print(space._concepts[concept].membership_of(point))
        print()'''
    if supercategory=='none':
        superkey = max(space._concepts,key= lambda candidate:space._concepts[candidate].membership_of(point))
    else:
        superkey = supercategory
    
    supercategory = space._concepts[superkey]
    avg = lambda values: sum(values)/len(values) if len(values) else float('inf')
    #print([[cuboid._p_max[dim]-cuboid._p_min[dim] for cuboid in supercategory._core._cuboids if not cuboid._p_max[dim]==float('inf')]for dim in range(space._n_dim)])
    avg_sizes=[avg([cuboid._p_max[dim]-cuboid._p_min[dim] for cuboid in supercategory._core._cuboids if not cuboid._p_max[dim]==float('inf')]) for dim in range(space._n_dim)]
    print('supercategory is:')
    print(superkey)
    print(avg_sizes)
    print()
    p_min = [point[i]-1/2*avg_sizes[i] for i in range(space._n_dim)]
    p_max = [point[i]+1/2*avg_sizes[i] for i in range(space._n_dim)]
    cuboid=Cuboid(p_min, p_max, space._domains)#not working in bigger examples
    core=Core([cuboid],space._domains)
    weights=Weights(space._def_dom_weights,space._def_dim_weights)
    concept=Concept(core, 1.0, 0.5, weights)
    return concept
        
''

''

In [3]:
#%matplotlib widget
concepts = xml_to_dict(path)
concepts = xml_to_dict(path2, concepts)

#print(concepts)

domains = {}

for concept in concepts.values():
    domains.update(concept)
del(domains['supercategory'])


example='bear'
        
to_learn={example:concepts[example]}
del(concepts[example])
mammal={key:concepts[key] for key in concepts if concepts[key]['supercategory']=='mammal'}

to_del=[]
for concept in concepts:
    if concepts[concept]['supercategory']=='object':# or concepts[concept]['supercategory']=='amphibian':
        to_del.append(concept)
for conc in to_del:
    del(concepts[conc])
        
concepts_into_space(concepts.copy(), except_for=to_learn[example]['supercategory'])

#print(space._concepts['reptile'])

dimension_values=[]
concepts=to_learn
consistent = True
#print(bear)
for domain in space._domains.keys():
    dimension_values.append([list(concepts[example].get(domain,{str(key):float("-inf") for key in range(len(space._domains[domain]))}).values())
                         ,list(concepts[example].get(domain,{str(key):float("inf") for key in range(len(space._domains[domain]))}).values())])
    if(not(len(dimension_values[-1][0])==len(space._domains[domain]))):
        print(domain+' in '+example+' is inconsistent')
        consistent=False
        break

domains = {}
for key in concepts[example]:
    if(not(key=='supercategory')):
        domains[key]=space._domains[key]

'''print(dimension_values)
print(domains)'''

'''if(consistent):
        p_min=[value for domain in dimension_values for value in domain[0]]
        p_max=[value for domain in dimension_values for value in domain[1]]

        #try:
        c_example = Cuboid(p_min, p_max, domains)
        s_example = Core([c_example], domains)
        w_example = Weights(space._def_dom_weights,space._def_dim_weights)
        bear = Concept(s_example, 1.0, 0.5, w_example)
        space.add_concept(example, bear, 'yellow')'''

'''for concept in space._concepts:'''
#print(concept)
p_max=[value for domain in dimension_values for value in domain[1]]
print('bear:')
print(p_max)

space.add_concept(example+' from membership',one_shot(p_max),'forestgreen')
space.add_concept(example+' from animal',one_shot(p_max,supercategory='animal'),'lime')

example='camel'

to_learn={example:mammal[example]}
dimension_values=[]
concepts=to_learn
consistent = True
#print(bear)
for domain in space._domains.keys():
    dimension_values.append([list(concepts[example].get(domain,{str(key):float("-inf") for key in range(len(space._domains[domain]))}).values())
                         ,list(concepts[example].get(domain,{str(key):float("inf") for key in range(len(space._domains[domain]))}).values())])
    if(not(len(dimension_values[-1][0])==len(space._domains[domain]))):
        print(domain+' in '+example+' is inconsistent')
        consistent=False
        break

domains = {}
for key in concepts[example]:
    if(not(key=='supercategory')):
        domains[key]=space._domains[key]
        
p_max=[value for domain in dimension_values for value in domain[1]]        
space.add_concept(example+' from membership',one_shot(p_max),'yellow')
space.add_concept(example+' from animal',one_shot(p_max,supercategory='animal'),'gold')

concepts=mammal
ordered_dict={}
for example in concepts:
    if concepts[example]['supercategory'] in ordered_dict:
        ordered_dict[concepts[example]['supercategory']].append(example)
    else:
        ordered_dict[concepts[example]['supercategory']]=[example]
for category in ordered_dict:
    print('putting',category,'into space')
    cuboids=[]
    domains = {}
    for example in ordered_dict[category]:
        if not is_consistent(concepts[example]):
            print(example,' is inconsistent')
        else:
            subdomains={domain:space._domains[domain] for domain in concepts[example] if not domain == 'supercategory'}
            point=[concepts[example].get(domain,{str(key):float("inf") for key in range(len(space._domains[domain]))}).values() for domain in space._domains]
            point=[p for dom in point for p in dom]
            #print(example)
            #print('point: ',point)
            cuboids.append(Cuboid([dim if not dim==float('inf') else -dim for dim in point], point, subdomains))
            domains.update(subdomains)
            #print(cuboids[-1]._p_max)
    #for cuboid in cuboids:
        #print('cuboid:',cuboid)
    core=from_cuboids(cuboids,domains)
    weights=Weights(space._def_dom_weights,space._def_dim_weights)
    concept=Concept(core, 1.0, 0.5, weights)
    space.add_concept(category,concept,'orange')
    
    
        #print(space._concepts[concept].membership_of([p if not p==float('-inf') else 0 for p in p_min]))
        #print(space._concepts[concept].membership_of(p_max))
            #print(bear.subset_of(space._concepts[concept]))
print('amphibian:')
print(space._concepts['amphibian'])
ci.init()

putting bird into space
putting insect into space
putting arthropod into space
putting reptile into space
putting amphibian into space
putting fish into space
putting mollusk into space
putting arachnid into space
bear:
[150.0, 250.0, 100.0]
supercategory is:
bird
[17.5, 49.166666666666664, 33.333333333333336]

supercategory is:
animal
[25.997959183673466, 57.63186813186813, 66.47755102040817]

supercategory is:
bear from animal
[25.99795918367346, 57.631868131868146, 66.47755102040816]

supercategory is:
animal
[25.997959183673466, 57.63186813186813, 66.47755102040817]

putting mammal into space
amphibian:
core: {[10.0, 15.0, 5.0]-[10.0, 20.0, 10.0], [10.0, 10.0, 10.0]-[10.0, 15.0, 15.0]}
mu: 1.0
c: 0.5
weights: <{'size': 1.0},{'size': {0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}}>


In [5]:
axis_ranges=[(-119.52000000000001, 719.9200000000001), (-108.80000000000001, 659.8), (-2399.52, 14399.92)]


cuboid=([-119.53000000000002, -108.81000000000002, -2399.53], [719.9300000000001, 659.81, 14399.93])
d1,d2=0,1
print(any([not (cuboid[0][i]==axis_ranges[i][0] and cuboid[1][i]==axis_ranges[i][1]) for i in [d1,d2]]))

True


In [6]:
cuboid=[[],[1,2,3,float('inf')]]

[cuboid[1][i]==float('inf') for i in [0,1,3]]

[False, False, True]

In [4]:
x={a:1 for a in {'1':10,'2':20,'3':30} if not a==3}
concepts={'example':{'supercategory':1,'locomotion':2,'size':3}}
example='example'
subdomains={domain:1 for domain in concepts[example] if not domain == 'supercategory'}
subdomains

{'locomotion': 1, 'size': 1}

In [3]:
min([1,2,None,3])

TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [3]:
bird
c1:  4.0  -  4.0  c2:  6.0  -  8.0 

c1:  10.0  -  10.0  c2:  16.875  -  25.66801525297619 

c1:  -inf  -  inf  c2:  -inf  -  inf 

c1:  -inf  -  inf  c2:  -inf  -  inf 

c1:  -inf  -  inf  c2:  -inf  -  inf 

c1:  -inf  -  inf  c2:  -inf  -  inf 

c1:  -inf  -  inf  c2:  -inf  -  inf 

c1:  -inf  -  inf  c2:  -inf  -  inf 

c1:  150.0  -  150.0  c2:  29.981498794987566  -  60.0 

c1:  250.0  -  250.0  c2:  55.51736111111111  -  100.0 

c1:  100.0  -  100.0  c2:  51.063169963987825  -  120.0 

c1:  30.0  -  30.0  c2:  42.45482653423327  -  100.0 

c1:  10.0  -  10.0  c2:  0.0  -  14.333333333333334 

c1:  40.0  -  40.0  c2:  0.0  -  23.333333333333332 

[  5.          13.4375     105.         175.         100.
  36.22741327  10.          31.66666667]
Singular matrix C in LSQ subproblem
0.0

bla
blu


In [3]:
a=float('inf')-float('inf')
a

nan

In [14]:
import xml.etree.ElementTree as ET
import os
import sys
sys.path.append("ConceptualSpaces-pure\\conceptual_spaces")
import cs.cs as space
from cs.weights import Weights
from cs.cuboid import Cuboid
from cs.core import Core
from cs.concept import Concept
import visualization.concept_inspector as ci


domains = {"color":[0], "shape":[1,2]}
dimension_names = ["hue", "round", 'something']
space.init(3, domains, dimension_names)



# standard weights for the dimensions within each domain
w_dim = {"shape":{1:1,2:1}}

# define pear concept
c_pear = Cuboid([float('-inf'),0.0, 0.0], [float('inf'),4.0, 10.0], {"shape":[1,2]})
s_pear = Core([c_pear], domains)
w_pear = Weights({"shape":0.8}, w_dim)
pear = Concept(s_pear, 1.0, 24.0, w_pear)
space.add_concept("pear", pear, 'g')

#del(domains['taste'])
w_dim.update({"color":{0:1}})

# define orange concept
c_orange = Cuboid([0.0,0.0,0.0], [5.0,5.0,10.0], domains)
s_orange = Core([c_orange,c_pear], domains)
w_orange = Weights({"color":1.2, "shape":0.8}, w_dim)
orange = Concept(s_orange, 1.0, 24.0, w_orange)
space.add_concept("orange", orange, 'orange')

print(pear.membership_of([1,1,12]))
print(orange.membership_of([1,1,12]))

#ci.init()

[inf, 1, 10.0] 
zu
[1, 1, 12] 
with
<{'shape': 1.0},{'shape': {1: 0.5, 2: 0.5}}> 

distance:  1.4142135623730951 

1.8178434809396932e-15
[1, 1, 10.0] 
zu
[1, 1, 12] 
with
<{'color': 1.2, 'shape': 0.8},{'shape': {1: 0.5, 2: 0.5}, 'color': {0: 1.0}}> 

distance:  1.1313708498984762 

[inf, 1, 10.0] 
zu
[1, 1, 12] 
with
<{'shape': 1.0},{'shape': {1: 0.5, 2: 0.5}, 'color': {0: 1.0}}> 

distance:  1.4142135623730951 

1.6130401952059783e-12


In [3]:
import xml.etree.ElementTree as ET
import os
import sys
sys.path.append("ConceptualSpaces-pure\\conceptual_spaces")
import cs.cs as space
from cs.weights import Weights
from cs.cuboid import Cuboid
from cs.core import Core
from cs.concept import Concept
import visualization.concept_inspector as ci


domains = {"locomotion": [0], "bla": [1]}
dimension_names = ["movement", "speed"]
space.init(2, domains, dimension_names)

inf = float('inf')

# standard weights for the dimensions within each domain
w_dim = {"locomotion":{0:1}, "bla":{1:1}}


# define pear concept
c_pear = Cuboid([0.4, 0.1], [0.5, 0.11], domains)
s_pear = Core([c_pear], domains)
w_pear = Weights({"locomotion": 1.0, "bla": 1.0}, w_dim)
pear = Concept(s_pear, 1.0, 24.0, w_pear)
space.add_concept("pear", pear, 'g')


# define orange concept
c_orange = Cuboid([0.6, 4.0], [0.8, 20.0], domains)
s_orange = Core([c_orange], domains)
w_orange = Weights({'locomotion': 1.0, 'bla': 1.0}, w_dim)
orange = Concept(s_orange, 1.0, 24.0, w_orange)
space.add_concept("orange", orange, 'orange')

print(orange.subset_of(pear))

#ci.init()

Singular matrix C in LSQ subproblem


Exception: Optimizer failed!

In [ ]:
import xml.etree.ElementTree as ET
import os
import sys
sys.path.append("ConceptualSpaces-pure\\conceptual_spaces")
import cs.cs as space
from cs.weights import Weights
from cs.cuboid import Cuboid
from cs.core import Core
from cs.concept import Concept
import visualization.concept_inspector as ci


domains = {'locomotion': [0, 1], 'location': [2, 3, 4, 5], 'feeding': [6, 7], 'size': [8, 9, 10], 'color': [11, 12, 13]}
dimension_names = ['movement', 'speed', 'humidity', 'temperature', 'altitude', 'vegetation', 'foodType', 'foodQuantity', 'x', 'y', 'z', 'l', 'a', 'b']
space.init(14, domains, dimension_names)

inf = float('inf')

# standard weights for the dimensions within each domain
w_dim = space._def_dim_weights
w_dom = space._def_dom_weights


domainsp=domains.copy()
del(domainsp['location'])
del(domainsp['feeding'])
# define pear concept
c_pear = Cuboid([4.0, 10.0, -inf, -inf, -inf, -inf, -inf, -inf, 150.0, 250.0, 100.0, 30.0, 10.0, 40.0], [4.0, 10.0, inf, inf, inf, inf, inf, inf, 150.0, 250.0, 100.0, 30.0, 10.0, 40.0], domainsp)
s_pear = Core([c_pear], domainsp)
w_pear = Weights(w_dom, w_dim)
pear = Concept(s_pear, 1.0, 24.0, w_pear)
space.add_concept("pear", pear, 'g')


# define orange concept
c_orange = Cuboid([6.0, 16.875, -inf, -inf, -inf, -inf, -inf, -inf, 29.981498794987566, 55.51736111111111, 51.063169963987825, 42.45482653423327, 0.0, 0.0], [8.0, 25.66801525297619, inf, inf, inf, inf, inf, inf, 60.0, 100.0, 120.0, 100.0, 14.333333333333334, 23.333333333333332], domainsp)
s_orange = Core([c_orange], domainsp)
w_orange = Weights(w_dom, w_dim)
orange = Concept(s_orange, 1.0, 30.0, w_orange)
space.add_concept("orange", orange, 'orange')

print(orange.subset_of(pear))

ci.init()